# 1 Creating a DataFrame

First, let's create some DataFrame from Python objects. While this is probably not the most common thing to do, it is easy and helpful in some situations where you already have some Python objects.

In [ ]:
# YOUR CODE HERE

## 1.1 Inspect Schema

The `spark` object has different methods for creating a so called Spark DataFrame object. This object is similar to a table, it contains rows of records, which all conform to a common schema with named columns and specific types. On the surface it heavily borrows concepts from Pandas DataFrames or R DataFrames, although the syntax and many operations are syntactically very different.

As the first step, we want to see the contents of the DataFrame. This can be easily done by using the show method.

In [ ]:
# YOUR CODE HERE

# 2 Reading Data

Of course manually creating DataFrames from a couple of records is not the real use case. Instead we want to read data frames files.. Spark supports various file formats, we will use JSON in the following example.

The entrypoint for creating Spark objects is an object called spark which is provided in the notebook and read to use. We will read a file containing some informations on a couple of persons, which will serve as the basis for the next examples

In [ ]:
# YOUR CODE HERE

In [ ]:
persons.collect()

## 2.1 Inspecting a DataFrame

Spark supports various methods for inspecting both the contents and the schema of a DataFrame

In [ ]:
# YOUR CODE HERE

In [ ]:
# YOUR CODE HERE

## Pandas Interoperability

Spark also supports interoperation with Python Pandas, the standard library for modelling tabular data.

In [ ]:
# YOUR CODE HERE

## 2.2 Loading CSV Data

Of course Spark also supports reading CSV data. CSV files may optionally contain a header containing the column names.

In [ ]:
# YOUR CODE HERE

# 3 Simple Transformations

## 3.1 Projections

The simplest thing to do is to create a new DataFrame with a subset of the available columns

In [ ]:
from pyspark.sql.functions import *

# YOUR CODE HERE

## 3.2 Addressing Columns

Spark supports multiple different ways for addressing a columns. We just saw one way, but also the following methods are supported for specifying a column:

* `df.column_name`
* `df['column_name']`
* `col('column_name')`

All these methods return a Column object, which is an abstract representative of the data in the column. As we will see soon, transformations can be applied to Column in order to derive new values.

### Beware of Lowercase and Uppercase

While PySpark itself is case insenstive concering column names, Python itself is case sensitive. Since the first method for addressing columns by treating them as fields of a Python object *is* Python syntax, this is also case sensitive!

In [ ]:
from pyspark.sql.functions import *

# YOUR CODE HERE

## 3.3 Transformations 

The `select` method actually accepts any column object. A column object conceptually represents a column in a DataFrame. The column may either refer directly to an existing column of the input DataFrame, or it may represent the result of a calculation or transformation of one or multiple columns of the input DataFrame. For example if we simply want to transform the name into upper case, we can do so by using a function `upper` provided by PySpark.

In [ ]:
# YOUR CODE HERE

### Defining new Column Names
The resulting DataFrame again has a schema, but the column names to not look very nice. But by using the `alias` method of a `Column` object, you can immediately rename the newly created column like you are already used to in SQL with `SELECT complex_operation(...) AS nice_name FROM ...`. 

Technically specifying a new name for the resulting column is not required (as we already saw above), if the name is not specified, PySpark will generate a name from the expression. But since this generated name tends to be rather long and contains the logic instead of the intention, it is highly recommended to always explicitly specify the name of the resulting column using `as`.

In [ ]:
# Result should be "Alice is 23 years old"
result = persons.select(
            concat(persons.name, lit(" is "), persons.age, lit(" years old")).alias("description")
        )
result.toPandas()

You can also perform simple mathematical calculations like addition, multiplication etc.

In [ ]:
result = persons.select((persons.age * 2).alias("age2"))
result.toPandas()

### Common Functions

You can find the full list of available functions at [PySpark SQL Module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions). Commonly used functions for example are as follows:

* [`concat(*cols)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.concat) - Concatenates multiple input columns together into a single column.
* [`substring(col,start,len)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.substring) - Substring starts at pos and is of length len when str is String type or returns the slice of byte array that starts at pos in byte and is of length len when str is Binary type.
* [`instr(col,substr)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.instr) - Locate the position of the first occurrence of substr column in the given string. Returns null if either of the arguments are null.
* [`locate(col,substr, pos)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.locate) - Locate the position of the first occurrence of substr in a string column, after position pos.
* [`length(col)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.length) - Computes the character length of string data or number of bytes of binary data. 
* [`upper(col)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.upper) - Converts a string column to upper case.
* [`lower(col)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.lower) - Converts a string column to lower case.
* [`coalesce(*cols)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.coalesce) - Returns the first column that is not null.
* [`isnull(col)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.isnull) - An expression that returns true iff the column is null.
* [`isnan(col)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.isnan) - An expression that returns true iff the column is NaN.
* [`hash(cols*)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.hash) - Calculates the hash code of given columns.

Spark also supports conditional expressions, like the SQL `CASE WHEN` construct
* [`when(condition, value)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.when) - Evaluates a list of conditions and returns one of multiple possible result expressions.

There are also some special functions often required
* [`col(str)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.col) - Returns a Column based on the given column name.
* [`lit(val)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.lit) - Creates a Column of literal value.
* [`expr(str)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.expr) - Parses the expression string into the column that it represents

### User Defined Functions
Unfortunately you cannot directly use normal Python functions for transforming DataFrame columns. Although PySpark already provides many useful functions, this might not always sufficient. But fortunately you can *convert* a standard Python function into a PySpark function, thereby defining a so called *user defined function* (UDF). Details will be explained in detail in the training.

## 3.4 Adding Columns

A special variant of a `select` statement is the `withColumn` method. While the `select` statement requires all resulting columns to be defined in as arguments, the `withColumn` method keeps all existing columns and adds a new one. This operation is quite useful since in many cases new columns are derived from the existing ones, while the old ones still should be contained in the result.

Let us have a look at a simple example, which only adds the salutation as a new column:

In [ ]:
# YOUR CODE HERE

As you can see from the example above, `withColumn` always takes two arguments: The first one is the name of the new column (and it has to be a string), and the second argument is the expression containing the logic for calculating the actual contents.

## 3.5 Dropping a Column

PySpark also supports the opposite operation which simply removes some columns from a dataframe. This is useful if you need to remove some sensitive data before saving it to disk:

In [ ]:
# YOUR CODE HERE

# 4 Filtering

*Filtering* denotes the process of keeping only rows which meet a certain filter criteria.

## 4.1 Simple `WHERE` clauses

PySpark support two different approaches. The first approach specifies the filtering expression as a PySpark expression using columns:

In [ ]:
result = # YOUR CODE HERE
result.toPandas()

You can also specify multiple conditions and combine them via logical operatios (`&` and `|`).

In [ ]:
result = persons.where((persons.age > 22) & (persons.height > 160))
result.toPandas()

The second approach simply uses a string containing an SQL expression:

In [ ]:
result = # YOUR CODE HERE
result.toPandas()

## 4.2 Limit Operations

When working with large datasets, it may be helpful to limit the amount of records (like an SQL `LIMIT` operation).

In [ ]:
result = # YOUR CODE HERE
result.toPandas()

# 5 Simple Aggregations

PySpark supports simple global aggregations, like `COUNT`, `MAX`, `MIN` etc...

In [ ]:
# YOUR CODE HERE

In [ ]:
result = # YOUR CODE HERE
result.toPandas()

# 6 Grouping & Aggregating

An important class of operation is grouping and aggregation, which is equivalnt to an SQL `SELECT aggregation GROUP BY grouping` statement. In PySpark, grouping and aggregation is always performed by first creating groups using `groupBy` immediately followed by aggregation expressions inside an `agg` method. (Actually there are also some predefined aggregations which can be used instead of `agg`, but they do not offer the flexiviliby which is required most of the time).

Note that in the `agg` method you only need to specify the aggregation expression, the grouping columns are added automatically by PySpark to the resulting DataFrame.

In [ ]:
# YOUR CODE HERE

## Aggregation Functions

PySpark supports many aggregation functions, they can be found in the documentation at [PySpark Function Documentation](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions). Aggregation functions are marked as such in the documentation, unfortunately there is no simple overview. Among common aggregation functions, there are for example:

* count
* sum
* avg
* corr
* first
* last

# 7 Sorting DataFrames

You can sort the entries (= rows) of a DataFrame by an arbitrary column or expression.

In [ ]:
result = # YOUR CODE HERE
result.toPandas()

If nothing else is specified, PySpark will sort the records in increasing order of the sort columns. If you require descending order, this can be specified by manipulating the sort column with the `desc()` method as follows:

In [ ]:
result = # YOUR CODE HERE
result.toPandas()

# User Defined Functions

Sometimes the built in functions do not suffice or you want to call an existing function of a Python library. Using User Defined Functions (UDF) it is possible to wrap an existing function into a Spark DataFrame function.

In [ ]:
import html
from pyspark.sql.types import *

html_encode = # YOUR CODE HERE

df = spark.createDataFrame([
        ("Alice & Bob",),
        ("Thelma & Louise",)
    ], ["name"])

result = # YOUR CODE HERE
result.toPandas()